In [14]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

In [15]:
df = pd.read_csv('schedulenets.csv')

In [16]:
df['time'] = pd.to_datetime(df['updatedAt'])

df['Date'] = df['time'].dt.date
df['Time'] = df['time'].dt.strftime('%H:%M')
df['Voltage']=df['block_1']

df = df[[ 'Date', 'Time','Voltage']]
df

,Date,Time,Voltage
0,2018-11-06,17:31,372.000000
1,2018-11-05,12:53,372.000000
2,2018-11-12,17:16,373.000000
3,2018-11-11,16:58,373.000000
4,2018-11-13,16:52,412.874368
...,...,...,...
10538,2021-06-11,09:30,200.000000
10539,2021-06-11,04:27,546.000000
10540,2021-06-11,06:07,466.000000
10541,2021-06-11,08:30,546.000000


In [17]:
df['Time_Intervals'] = df['Time'].apply(lambda x: f"{x[:2]}:{int(x[3:]) // 15 * 15:02d} to {x[:2]}:{(int(x[3:]) // 15 + 1) * 15:02d}")
df['Average_Voltage'] = df.groupby('Time_Intervals')['Voltage'].transform('mean')

df = df.groupby(['Date', 'Time_Intervals']).agg({'Average_Voltage': 'mean'}).reset_index()

data = df[[ 'Date', 'Time_Intervals','Average_Voltage']]
# df.to_csv('voltage_with_Intervals.csv', index=False) 
data

,Date,Time_Intervals,Average_Voltage
0,2018-11-05,12:45 to 12:60,417.272665
1,2018-11-05,17:30 to 17:45,407.169412
2,2018-11-06,17:30 to 17:45,407.169412
3,2018-11-10,16:45 to 16:60,386.672865
4,2018-11-11,16:45 to 16:60,386.672865
...,...,...,...
6540,2021-06-11,04:15 to 04:30,391.285290
6541,2021-06-11,06:00 to 06:15,401.741830
6542,2021-06-11,08:30 to 08:45,401.910695
6543,2021-06-11,09:00 to 09:15,397.025748


In [18]:
# Assuming your data is stored in a DataFrame named 'data'
data['Date'] = pd.to_datetime(data['Date'])

In [38]:
# Extract hours and minutes from 'Time_Intervals'
time_intervals = data['Time_Intervals'].str.extract(r'(\d+):(\d+) to (\d+):(\d+)')
data['Hour_Start'] = time_intervals[0].astype(int)
data['Minute_Start'] = time_intervals[1].astype(int)
data['Hour_End'] = time_intervals[2].astype(int)
data['Minute_End'] = time_intervals[3].astype(int)

# Convert 'Time_Start' and 'Time_End' to timedelta
data['Time_Start'] = pd.to_timedelta(data['Hour_Start'], unit='h') + pd.to_timedelta(data['Minute_Start'], unit='m')
data['Time_End'] = pd.to_timedelta(data['Hour_End'], unit='h') + pd.to_timedelta(data['Minute_End'], unit='m')

# Calculate the start and end times
data['Time_Interval_Start'] = data['Date'] + data['Time_Start']
data['Time_Interval_End'] = data['Date'] + data['Time_End']
data['Time_Interval_End']

0      2018-11-05 13:00:00
1      2018-11-05 17:45:00
2      2018-11-06 17:45:00
3      2018-11-10 17:00:00
4      2018-11-11 17:00:00
               ...        
6540   2021-06-11 04:30:00
6541   2021-06-11 06:15:00
6542   2021-06-11 08:45:00
6543   2021-06-11 09:15:00
6544   2021-06-11 09:45:00
Name: Time_Interval_End, Length: 6545, dtype: datetime64[ns]

In [20]:
# Extract relevant features
data['Hour'] = data['Time_Interval_Start'].dt.hour
data['Minute'] = data['Time_Interval_Start'].dt.minute
data['DayOfWeek'] = data['Time_Interval_Start'].dt.dayofweek
data['15_Min_Slot_Index'] = (data['Hour'] * 60 + data['Minute']) // 15

In [21]:
# Use only the necessary columns for training
X = data[['Hour', 'Minute', 'DayOfWeek', '15_Min_Slot_Index']].values
y = data['Average_Voltage'].values

In [22]:
# Normalize the features
scaler_X = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

In [23]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

In [24]:
# Reshape the input data for the LSTM model
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [25]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=50, return_sequences=True))
model.add(LSTM(units=50))
model.add(Dense(units=1))

In [26]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [27]:
# Fit the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/100
164/164 [==============================] - 8s 13ms/step - loss: 0.0684 - val_loss: 0.0324
Epoch 2/100
164/164 [==============================] - 1s 6ms/step - loss: 0.0297 - val_loss: 0.0309
Epoch 3/100
164/164 [==============================] - 1s 6ms/step - loss: 0.0291 - val_loss: 0.0309
Epoch 4/100
164/164 [==============================] - 1s 6ms/step - loss: 0.0289 - val_loss: 0.0305
Epoch 5/100
164/164 [==============================] - 1s 6ms/step - loss: 0.0289 - val_loss: 0.0304
Epoch 6/100
164/164 [==============================] - 1s 6ms/step - loss: 0.0286 - val_loss: 0.0301
Epoch 7/100
164/164 [==============================] - 1s 7ms/step - loss: 0.0285 - val_loss: 0.0303
Epoch 8/100
164/164 [==============================] - 1s 7ms/step - loss: 0.0279 - val_loss: 0.0293
Epoch 9/100
164/164 [==============================] - 1s 7ms/step - loss: 0.0274 - val_loss: 0.0280
Epoch 10/100
164/164 [==============================] - 1s 6ms/step - loss: 0.0261 - val_l

In [36]:
# Make predictions for the 1st, 6th, and 11th minute of each 15-minute output interval
predictions = model.predict(X_test)
predictions_rescaled = scaler_y.inverse_transform(predictions.reshape(-1, 1))

41/41 [==============================] - 3s 4ms/step


In [ ]:
print("Predicted energy output demand:")
for i, interval in enumerate([1, 6, 11]):
    print(f"{interval}th Minute: {predictions_rescaled[i][0]:.2f}")